In [1]:
import pytz, datetime, os
from urllib.parse import quote

query = '''
with email_base as (
  select uuid, 
    raw_data,
    array(select trim(email, '"') 
          from unnest(JSON_EXTRACT_ARRAY(REGEXP_EXTRACT(processed_data, r'(\[[^\]]*\])'), '$')) as email
          where regexp_contains(email, r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+')) as email_list
  from `greenlabs-data-farmmorning.content_analysis_us.gs_crawling_packer_info_processed`
  where source_id = "gs://greenlabs-data-grainscanner/crawling/raw/(그레인스캐너-데이터팀) 패커 크롤링 - Gulfood_1차 정리.csv"
    and data_type = 'email'
    and job_type = 'LLM'
  qualify row_number() over(partition by uuid order by process_ts desc) = 1
)
, email as (
  select uuid
      , email_list[safe_offset(0)] as new_main_email
      , (select string_agg(email, ', ') 
         from unnest(email_list) as email 
         where email <> email_list[safe_offset(0)]) as new_other_email
  from email_base
  where array_length(email_list) > 0
)
, address as (
  select uuid, 
    processed_data as new_contry
  from `greenlabs-data-farmmorning.content_analysis_us.gs_crawling_packer_info_processed`
  where source_id = "gs://greenlabs-data-grainscanner/crawling/raw/(그레인스캐너-데이터팀) 패커 크롤링 - Gulfood_1차 정리.csv"
    and data_type = 'address'
    and job_type = 'RESULT'
  qualify row_number() over(partition by uuid order by process_ts desc) = 1
)
, sku_base as (
  select uuid, 
    raw_data,
    array(select sku from unnest(JSON_EXTRACT_STRING_ARRAY(processed_data, '$')) as sku) as sku_list
  from `greenlabs-data-farmmorning.content_analysis_us.gs_crawling_packer_info_processed`
  where source_id = "gs://greenlabs-data-grainscanner/crawling/raw/(그레인스캐너-데이터팀) 패커 크롤링 - Gulfood_1차 정리.csv"
    and data_type = 'SKU'
    and job_type = 'RESULT'
  qualify row_number() over(partition by uuid order by process_ts desc) = 1
)
, sku as (
  select uuid
    , sku_list[safe_offset(0)] as new_sku1
    , sku_list[safe_offset(1)] as new_sku2
    , sku_list[safe_offset(2)] as new_sku3
    , sku_list[safe_offset(3)] as new_sku4
    , sku_list[safe_offset(4)] as new_sku5
    , sku_list[safe_offset(5)] as new_sku6
    , sku_list[safe_offset(6)] as new_sku7
    , sku_list[safe_offset(7)] as new_sku8
  from sku_base
  where array_length(sku_list) > 0
)
select raw.* except(source_id, uuid, process_ts)
    , email.* except(uuid)
    , address.* except(uuid)
    , sku.* except(uuid)
from `greenlabs-data-farmmorning.content_analysis_us.gs_crawling_packer_info_raw` as raw
left outer join email
  on raw.uuid = email.uuid
left outer join address
  on raw.uuid = address.uuid
left outer join sku
  on raw.uuid = sku.uuid
where source_id = "gs://greenlabs-data-grainscanner/crawling/raw/(그레인스캐너-데이터팀) 패커 크롤링 - Gulfood_1차 정리.csv"
qualify row_number() over(partition by raw.uuid order by process_ts desc) = 1
'''

job_info = {
        "raw_table": "greenlabs-data-farmmorning.content_analysis_us.gs_crawling_packer_info_raw",
        "slack_channels": "C043E8W8NBZ",  # 데이터-모니터링-인포
        "schema_dict": {
                        "패커명*": "packer_name",
                        "패커명2": "packer_name2",
                        "연락단계*": "contact_stage",
                        "검증단계*": "verification_stage",
                        "획득경로1": "acquisition_source1",
                        "획득경로2": "acquisition_source2",
                        "획득경로3": "acquisition_source3",
                        "획득경로4": "acquisition_source4",
                        "획득경로5": "acquisition_source5",
                        "취급 SKU1": "handled_sku1",
                        "취급 SKU2": "handled_sku2",
                        "취급 SKU3": "handled_sku3",
                        "취급 SKU4": "handled_sku4",
                        "취급 SKU5": "handled_sku5",
                        "취급 SKU6": "handled_sku6",
                        "취급 SKU7": "handled_sku7",
                        "취급 SKU8": "handled_sku8",
                        "국가(영어로)": "country",
                        "홈페이지 주소": "website_url",
                        "패커 담당자": "packer_contact_person",
                        "전화번호": "phone_number",
                        "대표 이메일": "primary_email",
                        "참조 이메일(여러개 입력시 콤마(,)로 구분)": "reference_emails",
                        "중복여부 표시": "duplicate_indicator",
                        "기존 전화번호": "original_phone_number",
                        "SKU raw": "sku_raw",
                        "주소 raw": "address_raw",
                        "주소 작업대상": "address_processing_target",
                        "SKU 작업대상": "sku_processing_target",
                        "이메일 작업 대상": "email_processing_target"
                    },
    }
name = 'gs://greenlabs-data-grainscanner/crawling/raw/(그레인스캐너-데이터팀) 패커 크롤링 - Gulfood_1차 정리.csv'

kst = pytz.timezone("Asia/Seoul")
current_kst = datetime.datetime.now(kst)
current_kst = current_kst.strftime('%Y%m%d_%H%M%S')

directory, filename = os.path.split(name)
file_base, file_extension = os.path.splitext(filename)

file_name = f"{file_base}_{current_kst}{file_extension}"
gcs_path = f'crawling/processed/{file_name}'
url_gcs_path = f'crawling/processed/{quote(file_name)}'

# XCom을 통해 job_id 가져오기
from google.cloud import bigquery
bigquery_client = bigquery.Client(project='grainscanner')

# job_id를 사용하여 BigQuery 작업 가져오기
df = bigquery_client.query(query).to_dataframe()


In [3]:

reverse_schema = {value: key for key, value in job_info['schema_dict'].items()}
df = df.rename(columns=reverse_schema)


In [5]:
import csv 

df.to_csv('./output/error_test.csv', quoting=csv.QUOTE_ALL, index=False)

In [34]:
import uuid
import pandas as pd

new_df = pd.DataFrame([{"a": "123", "b": "123"}, {"a": "23", "b": "23"}
                       ,{"a": "123", "b": "123"}, {"a": "23", "b": "23"}
                       ,{"a": "123", "b": "123"}, {"a": "23", "b": "23"}
                       ,{"a": "123", "b": "123"}, {"a": "23", "b": "23"}
                       ,{"a": "123", "b": "123"}, {"a": "23", "b": "23"}
                       ,{"a": "123", "b": "123"}, {"a": "23", "b": "23"}
                       ,{"a": "123", "b": "123"}, {"a": "23", "b": "23"}])

import uuid
index_len = len(str(len(new_df)))
splitted_uuid = str(uuid.uuid4())[index_len+1:]
new_df['uuid'] = [('0000000' + str(i))[-index_len:] + '$' + splitted_uuid for i in range(1, len(new_df)+1)]
new_df['lllen'] = new_df['uuid'].apply(len)
    

In [35]:
new_df

,a,b,uuid,lllen
0,123,123,01$b3be7-42f1-414a-b96a-547c7b2e94e4,36
1,23,23,02$b3be7-42f1-414a-b96a-547c7b2e94e4,36
2,123,123,03$b3be7-42f1-414a-b96a-547c7b2e94e4,36
3,23,23,04$b3be7-42f1-414a-b96a-547c7b2e94e4,36
4,123,123,05$b3be7-42f1-414a-b96a-547c7b2e94e4,36
5,23,23,06$b3be7-42f1-414a-b96a-547c7b2e94e4,36
6,123,123,07$b3be7-42f1-414a-b96a-547c7b2e94e4,36
7,23,23,08$b3be7-42f1-414a-b96a-547c7b2e94e4,36
8,123,123,09$b3be7-42f1-414a-b96a-547c7b2e94e4,36
9,23,23,10$b3be7-42f1-414a-b96a-547c7b2e94e4,36
